# Audio Continuation Transformer Decoder (Train)

## Imports

In [1]:
import numpy as np
import math
import time
import csv
import os
import re
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as nnF
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchaudio.transforms as transforms
from collections import OrderedDict

from vocos import Vocos

import IPython
from IPython.display import display
import ipywidgets as widgets

## Settings

### Compute Device

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### Audio Settings

In [3]:
audio_file_path = "../../../../Data/Audio/Yannick_Leonie/48khz/"
audio_files = [] # audio files are automatically collected from the specified folder and added here
audio_sample_rate = 48000

audio_files_all = [f for f in os.listdir(audio_file_path) if os.path.isfile(os.path.join(audio_file_path, f))]

audio_file_path_gui = widgets.Text(str(audio_file_path), description='Audio File Path:', style={'description_width': 'initial'})
audio_sample_rate_gui = widgets.IntText(audio_sample_rate, description='Audio Sample Rate:', style={'description_width': 'initial'})

display(audio_file_path_gui)
display(audio_sample_rate_gui)

Text(value='../../../../Data/Audio/Yannick_Leonie/48khz/', description='Audio File Path:', style=TextStyle(des…

IntText(value=48000, description='Audio Sample Rate:', style=DescriptionStyle(description_width='initial'))

In [4]:
audio_file_path = audio_file_path_gui.value
audio_sample_rate = audio_sample_rate_gui.value

def gather_audio_files(root_dir, suffixes):
    
    if isinstance(suffixes, str):
        suffixes_tuple = (suffixes,)
    else:
        suffixes_tuple = tuple(suffixes)

    matches = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename.endswith(suffixes_tuple):
                matches.append(os.path.join(dirpath, filename))

    return matches

audio_files = gather_audio_files(audio_file_path, ["wav", "aif", "aiff"])

### Model Settings

In [5]:
decoder_layer_count = 6
decoder_head_count = 8
decoder_embed_dim = 512
decoder_ff_dim = 2048
decoder_dropout = 0.1

save_weights = True
load_weights = False
decoder_weights_file = "results/weights/decoder_weights_epoch_200"

decoder_layer_count_gui = widgets.IntText(value=decoder_layer_count, description="Decoder Layer Count:", style={'description_width': 'initial'})
decoder_head_count_gui = widgets.IntText(value=decoder_head_count, description="Decoder Head Count:", style={'description_width': 'initial'})
decoder_embed_dim_gui = widgets.IntText(value=decoder_embed_dim, description="Decoder Embed Dim:", style={'description_width': 'initial'})
decoder_ff_dim_gui = widgets.IntText(value=decoder_ff_dim, description="Decoder Fordward Dim:", style={'description_width': 'initial'})
decoder_dropout_gui = widgets.FloatText(value=decoder_dropout, description="Decoder Dropout:", style={'description_width': 'initial'})

save_weights_gui = widgets.Checkbox(
    value=save_weights,
    description='Save Weights',
    disabled=False,
    style={'description_width': 'initial'}
)

load_weights_gui = widgets.Checkbox(
    value=load_weights,
    description='Load Weights',
    disabled=False,
    style={'description_width': 'initial'}
)

decoder_weights_file_gui = widgets.Text(value=decoder_weights_file, description="Decoder Weights File:", style={'description_width': 'initial'}) 

display(decoder_layer_count_gui)
display(decoder_head_count_gui)
display(decoder_embed_dim_gui)
display(decoder_ff_dim_gui)
display(decoder_dropout_gui)
display(save_weights_gui)
display(load_weights_gui)
display(decoder_weights_file_gui)

IntText(value=6, description='Decoder Layer Count:', style=DescriptionStyle(description_width='initial'))

IntText(value=8, description='Decoder Head Count:', style=DescriptionStyle(description_width='initial'))

IntText(value=512, description='Decoder Embed Dim:', style=DescriptionStyle(description_width='initial'))

IntText(value=2048, description='Decoder Fordward Dim:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.1, description='Decoder Dropout:', style=DescriptionStyle(description_width='initial'))

Checkbox(value=True, description='Save Weights', style=CheckboxStyle(description_width='initial'))

Checkbox(value=False, description='Load Weights', style=CheckboxStyle(description_width='initial'))

Text(value='results/weights/decoder_weights_epoch_200', description='Decoder Weights File:', style=TextStyle(d…

In [6]:
decoder_layer_count = decoder_layer_count_gui.value
decoder_head_count = decoder_head_count_gui.value
decoder_embed_dim = decoder_embed_dim_gui.value
decoder_ff_dim = decoder_ff_dim_gui.value
decoder_dropout = decoder_dropout_gui.value
save_weights = save_weights_gui.value
load_weights = load_weights_gui.value
decoder_weights_file = decoder_weights_file_gui.value

### Training settings

In [7]:
batch_size = 32
test_percentage = 0.1

seq_input_length = 64
seq_output_length = 10 # this is only used for non-teacher forcing scenarios

learning_rate = 1e-4
teacher_forcing_prob = 0.0
model_save_interval = 10

epochs = 200
save_history = True

batch_size_gui = widgets.IntText(value=batch_size, description="Batch Size:", style={'description_width': 'initial'})
test_percentage_gui = widgets.FloatText(value=test_percentage, description="Test Ratio:", style={'description_width': 'initial'})
seq_input_length_gui = widgets.IntText(value=seq_input_length, description="Sequence Input Length:", style={'description_width': 'initial'})
seq_output_length_gui = widgets.IntText(value=seq_output_length, description="Sequence Output Length:", style={'description_width': 'initial'})
learning_rate_gui = widgets.FloatText(value=learning_rate, description="Autoencoder Learning Rate:", style={'description_width': 'initial'})
teacher_forcing_prob_gui = widgets.FloatText(value=teacher_forcing_prob, description="Teacher Forcing Probability:", style={'description_width': 'initial'})
model_save_interval_gui = widgets.IntText(value=model_save_interval, description="Model Save Interval:", style={'description_width': 'initial'})
epochs_gui = widgets.IntText(value=epochs, description="Epochs:", style={'description_width': 'initial'})

display(batch_size_gui)
display(test_percentage_gui)
display(seq_input_length_gui)
display(seq_output_length_gui)
display(learning_rate_gui)
display(teacher_forcing_prob_gui)
display(model_save_interval_gui)
display(epochs_gui)

IntText(value=32, description='Batch Size:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.1, description='Test Ratio:', style=DescriptionStyle(description_width='initial'))

IntText(value=64, description='Sequence Input Length:', style=DescriptionStyle(description_width='initial'))

IntText(value=10, description='Sequence Output Length:', style=DescriptionStyle(description_width='initial'))

FloatText(value=0.0001, description='Autoencoder Learning Rate:', style=DescriptionStyle(description_width='in…

FloatText(value=0.0, description='Teacher Forcing Probability:', style=DescriptionStyle(description_width='ini…

IntText(value=10, description='Model Save Interval:', style=DescriptionStyle(description_width='initial'))

IntText(value=200, description='Epochs:', style=DescriptionStyle(description_width='initial'))

In [8]:
batch_size = batch_size_gui.value
test_percentage = test_percentage_gui.value
seq_input_length = seq_input_length_gui.value
seq_output_length = seq_output_length_gui.value
learning_rate = learning_rate_gui.value
teacher_forcing_prob = teacher_forcing_prob_gui.value
model_save_interval = model_save_interval_gui.value
epochs = epochs_gui.value

## Load Audio Data and Calculate Mel Spectra

In [9]:
vocos = Vocos.from_pretrained("kittn/vocos-mel-48khz-alpha1")

all_audio_features = []

for audio_file_index in range(len(audio_files)):
    
    print("audio file ", audio_files[audio_file_index])
    
    audio_file = audio_files[audio_file_index]
    
    # load audio file
    waveform_data, _ = torchaudio.load(audio_file_path + audio_file)
    
    print("waveform_data s ", waveform_data.shape)
    
    # audio features
    audio_features = vocos.feature_extractor(waveform_data)
    
    print("audio_features s ", audio_features.shape)
    
    audio_features = audio_features.squeeze(0)
    audio_features = torch.permute(audio_features, (1, 0))
    
    print("audio_features 2 s ", audio_features.shape)
    
    all_audio_features.append(audio_features)

C:\Users\dbisig\anaconda3\envs\ima2025\lib\site-packages\vocos\pretrained.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_locati

audio file  ../../../../Data/Audio/Yannick_Leonie/48khz/BeeFlyBy_BU01.23.wav
waveform_data s  torch.Size([1, 296192])
audio_features s  torch.Size([1, 128, 1158])
audio_features 2 s  torch.Size([1158, 128])
audio file  ../../../../Data/Audio/Yannick_Leonie/48khz/DS_SCB_fx_ambience_one_shot_walking_forest.wav
waveform_data s  torch.Size([1, 1822400])
audio_features s  torch.Size([1, 128, 7119])
audio_features 2 s  torch.Size([7119, 128])
audio file  ../../../../Data/Audio/Yannick_Leonie/48khz/ESM_WNL_fx_birds_loop_song_musical_breeze_lively_forest_01.wav
waveform_data s  torch.Size([1, 5056768])
audio_features s  torch.Size([1, 128, 19754])
audio_features 2 s  torch.Size([19754, 128])
audio file  ../../../../Data/Audio/Yannick_Leonie/48khz/ESM_WNL_fx_water_flow_loop_stream_forest_creek_quiet_insects_01.wav
waveform_data s  torch.Size([1, 3799365])
audio_features s  torch.Size([1, 128, 14842])
audio_features 2 s  torch.Size([14842, 128])
audio file  ../../../../Data/Audio/Yannick_Leonie/

## Create Dataset

In [10]:
X = []
y = []

audio_features_dim = all_audio_features[0].shape[-1]

audio_features_dim

for audio_features in all_audio_features:
    
    total_sequence_length = audio_features.shape[0]
    
    for pI in range(total_sequence_length - seq_input_length - seq_output_length - 1):
        X_sample = audio_features[pI:pI+seq_input_length]
        X.append(X_sample)
        
        y_sample = audio_features[pI+seq_input_length:pI+seq_input_length+seq_output_length]
        y.append(y_sample)

X = np.array(X)
y = np.array(y)

X = torch.from_numpy(X)
y = torch.from_numpy(y)

class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx, ...], self.y[idx, ...]


full_dataset = SequenceDataset(X, y)

X_item, y_item = full_dataset[0]

print("X_item s ", X_item.shape)
print("y_item s ", y_item.shape)

test_size = int(test_percentage * len(full_dataset))
train_size = len(full_dataset) - test_size

train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X_batch, y_batch = next(iter(train_loader))

print("X_batch s ", X_batch.shape)
print("y_batch s ", y_batch.shape)

X_item s  torch.Size([64, 128])
y_item s  torch.Size([10, 128])
X_batch s  torch.Size([32, 64, 128])
y_batch s  torch.Size([32, 10, 128])


## Create Models

## PositionalEncoding

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

## Create TransformerDecoder

In [12]:
class TransformerDecoder(nn.Module):

    # Constructor
    def __init__(
        self,
        audio_dim,
        embed_dim,
        num_heads,
        num_decoder_layers,
        ff_dim,
        dropout_p,
        pos_encoding_max_length
    ):
        super().__init__()

        self.embed_dim = embed_dim

        self.audio2embed = nn.Linear(audio_dim, embed_dim) # map audio data to embedding

        self.positional_encoder = PositionalEncoding(
            dim_model=embed_dim, dropout_p=dropout_p, max_len=pos_encoding_max_length
        )
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout_p, batch_first=True)
        #self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers = num_decoder_layers)

        # build a decoder directly from TransformerDecoderLayer
        # rather than using the nn.TransformerDecoder module which requires also a Transformer Encoder
        self.decoder = self.layers = nn.ModuleList([
            nn.TransformerDecoderLayer(
                d_model=embed_dim,
                nhead=num_heads,
                dim_feedforward=ff_dim,
                dropout=dropout_p,
                activation='gelu',
                batch_first=True
            ) for _ in range(num_decoder_layers)
        ])

        self.embed2audio = nn.Linear(embed_dim, audio_dim) # map embedding to audio data

    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask
        
       
    def forward(self, audio_data):
        
        #print("forward")
        
        #print("audio_data s ", audio_data.shape)
        
        # dummy "memory" as zero (only self-attention is used)
        memory = torch.zeros(audio_data.size(0), audio_data.size(1), self.embed_dim, device=audio_data.device)

        #print("memory s ", memory.shape)

        # Lower triangular matrix for autoregressive masking
        tgt_mask = self.get_tgt_mask(audio_data.shape[1]).to(audio_data.device)

        #print("tgt_mask s ", tgt_mask.shape)

        audio_embedded = self.audio2embed(audio_data) * math.sqrt(self.embed_dim)
        
        #print("audio_embedded 1 s ", audio_embedded.shape)
        
        audio_embedded = self.positional_encoder(audio_embedded)
        
        #print("audio_embedded 2 s ", audio_embedded.shape)
        
        x = audio_embedded
        
        #print("x s ", x.shape)
        
        for layer in self.layers:
            
            #print("x in s ", x.shape)
            
            x = layer(x, memory, tgt_mask=tgt_mask)
            
            #print("x out s ", x.shape)

        decoder_out = x

        out = self.embed2audio(decoder_out)
        
        out = out[:, -1, :] # only last time step 
        
        return out

decoder = TransformerDecoder(audio_dim=audio_features_dim,
                          embed_dim=decoder_embed_dim, 
                          num_heads=decoder_head_count, 
                          num_decoder_layers=decoder_layer_count, 
                          ff_dim = decoder_ff_dim,
                          dropout_p=decoder_dropout,
                          pos_encoding_max_length=seq_input_length).to(device)

print(decoder)

if load_weights == True:
    decoder.load_state_dict(torch.load(decoder_weights_file))

# test transformer decoder
x_batch, _ = next(iter(train_loader))

decoder_input = x_batch.to(device)
decoder_output = decoder(decoder_input)

print("decoder_input s ", decoder_input.shape)
print("decoder_output s ", decoder_output.shape)


TransformerDecoder(
  (audio2embed): Linear(in_features=128, out_features=512, bias=True)
  (positional_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (decoder): ModuleList(
    (0-5): 6 x TransformerDecoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
      )
      (mult

## Training

### Create Optimizer and Scheduler

In [13]:
optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.336) # reduce the learning every 20 epochs by a factor of 10

### Create Loss Functions

In [14]:
rec_loss = nn.MSELoss()

def loss(y, yhat):
    _rec_loss = rec_loss(yhat, y)

    return _rec_loss

### Create Training Functions

In [15]:
def train_step(input_features, target_features, teacher_forcing):
    
    decoder.train()

    #print("ar_train_step")    
    #print("teacher_forcing ", teacher_forcing)
    #print("pose_sequences s ", pose_sequences.shape)
    #print("target_poses s ", target_poses.shape)

    #_input_poses = pose_sequences.detach().clone()    
    _input_features = input_features  
    output_features_length = target_features.shape[1]
    
    #print("output_features_length ", output_features_length)
    
    _pred_features_for_loss = []
    _target_features_for_loss = []
    
    for o_i in range(1, output_features_length):
        
        #print("_input_features s ", _input_features.shape)
        
        _pred_features = decoder(_input_features)
        _pred_features = torch.unsqueeze(_pred_features, axis=1)
        
        #print("_pred_features s ", _pred_features.shape)
        
        _target_features = target_features[:,o_i,:].detach().clone()
        _target_features = torch.unsqueeze(_target_features, axis=1)

        #print("_target_features s ", _target_features.shape)
        
        _pred_features_for_loss.append(_pred_features)
        _target_features_for_loss.append(_target_features)
        
        # shift input feature seqeunce one feature to the right
        # remove feature from beginning input feature sequence
        # detach necessary to avoid error concerning running backprob a second time
        _input_features = _input_features[:, 1:, :].detach().clone()
        _target_features = _target_features.detach().clone()
        _pred_features = _pred_features.detach().clone()
        
        # add predicted or target feature to end of input feature sequence
        if teacher_forcing == True:
            _input_features = torch.concat((_input_features, _target_features), axis=1)
        else:
            _input_features = torch.cat((_input_features, _pred_features), axis=1)
            
        #print("_input_features s ", _input_features.shape)

        
    _pred_features_for_loss = torch.cat(_pred_features_for_loss, dim=1)
    _target_features_for_loss = torch.cat(_target_features_for_loss, dim=1)
    
    #print("_pred_features_for_loss 2 s ", _pred_features_for_loss.shape)
    #print("_target_features_for_loss 2 s ", _target_features_for_loss.shape)
    
    _loss = loss(_target_features_for_loss, _pred_features_for_loss) 
    
    # Backpropagation
    optimizer.zero_grad()
    _loss.backward()
    optimizer.step()
    
    #print("_ar_loss_total mean s ", _ar_loss_total.shape)
    
    #return _ar_loss, _ar_norm_loss, _ar_quat_loss
    
    return _loss

def test_step(input_features, target_features, teacher_forcing):
    
    decoder.eval()

    #print("ar_train_step")    
    #print("teacher_forcing ", teacher_forcing)
    #print("pose_sequences s ", pose_sequences.shape)
    #print("target_poses s ", target_poses.shape)

    #_input_poses = pose_sequences.detach().clone()    
    _input_features = input_features  
    output_features_length = target_features.shape[1]
    
    #print("output_features_length ", output_features_length)
    
    _pred_features_for_loss = []
    _target_features_for_loss = []
    
    with torch.no_grad():
    
        for o_i in range(1, output_features_length):
            
            #print("_input_features s ", _input_features.shape)
            
            _pred_features = decoder(_input_features)
            _pred_features = torch.unsqueeze(_pred_features, axis=1)
            
            #print("_pred_features s ", _pred_features.shape)
            
            _target_features = target_features[:,o_i,:].detach().clone()
            _target_features = torch.unsqueeze(_target_features, axis=1)
    
            #print("_target_features s ", _target_features.shape)
            
            _pred_features_for_loss.append(_pred_features)
            _target_features_for_loss.append(_target_features)
            
            # shift input feature seqeunce one feature to the right
            # remove feature from beginning input feature sequence
            # detach necessary to avoid error concerning running backprob a second time
            _input_features = _input_features[:, 1:, :].detach().clone()
            _target_features = _target_features.detach().clone()
            _pred_features = _pred_features.detach().clone()
            
            # add predicted or target feature to end of input feature sequence
            if teacher_forcing == True:
                _input_features = torch.concat((_input_features, _target_features), axis=1)
            else:
                _input_features = torch.cat((_input_features, _pred_features), axis=1)
                
            #print("_input_features s ", _input_features.shape)

        
    _pred_features_for_loss = torch.cat(_pred_features_for_loss, dim=1)
    _target_features_for_loss = torch.cat(_target_features_for_loss, dim=1)
    
    #print("_pred_features_for_loss 2 s ", _pred_features_for_loss.shape)
    #print("_target_features_for_loss 2 s ", _target_features_for_loss.shape)
    
    _loss = loss(_target_features_for_loss, _pred_features_for_loss) 
    
    #print("_ar_loss_total mean s ", _ar_loss_total.shape)
    
    #return _ar_loss, _ar_norm_loss, _ar_quat_loss
    
    decoder.eval()
    
    return _loss

def train(train_dataloader, test_dataloader, epochs):
    
    loss_history = {}
    loss_history["train"] = []
    loss_history["test"] = []

    for epoch in range(epochs):
        start = time.time()
        
        _train_loss_per_epoch = []

        for train_batch in train_dataloader:
            input_feature_sequences = train_batch[0].to(device)
            target_features = train_batch[1].to(device)
            
            use_teacher_forcing = np.random.uniform() < teacher_forcing_prob
            
            _loss = train_step(input_feature_sequences, target_features, use_teacher_forcing)
            
            _loss = _loss.detach().cpu().numpy()
            
            _train_loss_per_epoch.append(_loss)

        _train_loss_per_epoch = np.mean(np.array(_train_loss_per_epoch))

        _test_loss_per_epoch = []
        
        for test_batch in test_dataloader:
            input_feature_sequences = train_batch[0].to(device)
            target_features = train_batch[1].to(device)
            
            use_teacher_forcing = np.random.uniform() < teacher_forcing_prob
            
            _loss = test_step(input_feature_sequences, target_features, use_teacher_forcing)
            
            _loss = _loss.detach().cpu().numpy()
            
            _test_loss_per_epoch.append(_loss)
        
        _test_loss_per_epoch = np.mean(np.array(_test_loss_per_epoch))
        
        if epoch % model_save_interval == 0 and save_weights == True:
            torch.save(decoder.state_dict(), "results/weights/decoder_weights_epoch_{}".format(epoch))
        
        loss_history["train"].append(_train_loss_per_epoch)
        loss_history["test"].append(_test_loss_per_epoch)
        
        scheduler.step()
        
        print ('epoch {} : train: {:01.4f} test: {:01.4f} time {:01.2f}'.format(epoch + 1, _train_loss_per_epoch, _test_loss_per_epoch, time.time()-start))
    
    return loss_history

### Run Training

In [ ]:
loss_history = train(train_loader, test_loader, epochs)

### Save Training Results

In [ ]:
def save_loss_as_image(loss_history, image_file_name):
    keys = list(loss_history.keys())
    epochs = len(loss_history[keys[0]])
    
    for key in keys:
        plt.plot(range(epochs), loss_history[key], label=key)
        
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(image_file_name)
    plt.show()

def save_loss_as_csv(loss_history, csv_file_name):
    with open(csv_file_name, 'w') as csv_file:
        csv_columns = list(loss_history.keys())
        csv_row_count = len(loss_history[csv_columns[0]])
        
        
        csv_writer = csv.DictWriter(csv_file, fieldnames=csv_columns, delimiter=',', lineterminator='\n')
        csv_writer.writeheader()
    
        for row in range(csv_row_count):
        
            csv_row = {}
        
            for key in loss_history.keys():
                csv_row[key] = loss_history[key][row]

            csv_writer.writerow(csv_row)


save_loss_as_csv(loss_history, "results/histories/rnn_history_{}.csv".format(epochs))
save_loss_as_image(loss_history, "results/histories/rnn_history_{}.png".format(epochs))

### Save Final Model Weights

In [ ]:
torch.save(decoder.state_dict(), "results/weights/decoder_weights_epoch_{}".format(epochs))

## Inference

In [ ]:
decoder.eval()

def export_orig_audio(waveform_data, start_time, end_time, file_name):
    
    start_time_samples = int(start_time * audio_sample_rate)
    end_time_samples = int(end_time * audio_sample_rate)
    
    torchaudio.save(file_name, waveform_data[:, start_time_samples:end_time_samples], audio_sample_rate)

def export_ref_audio(waveform_data, start_time, end_time, file_name):
    
    start_time_samples = int(start_time * audio_sample_rate)
    end_time_samples = int(end_time * audio_sample_rate)
    
    # audio features
    audio_features = vocos.feature_extractor(waveform_data[:, start_time_samples:end_time_samples])
    
    ref_audio = vocos.decode(audio_features)
    
    torchaudio.save(file_name, ref_audio.detach().cpu(), audio_sample_rate)
    

def export_pred_audio(waveform_data, start_time, end_time, file_name):
    
    start_time_samples = int(start_time * audio_sample_rate)
    end_time_samples = int(end_time * audio_sample_rate)
    
    # audio features
    audio_features = vocos.feature_extractor(waveform_data[:, start_time_samples:end_time_samples])
    
    #print("audio_features s ", audio_features.shape)
    
    audio_features = audio_features.squeeze(0)
    audio_features = torch.permute(audio_features, (1, 0))
    audio_feature_count = audio_features.shape[0]
    
    #print("audio_feature_count ", audio_feature_count)
    
    input_features = audio_features[:seq_input_length]
    input_features = input_features.unsqueeze(0)
    
    output_features_length = audio_feature_count - seq_input_length
    
    #print("output_features_length ", output_features_length)
    
    _input_features = input_features  
    pred_features = []
    
    with torch.no_grad():
    
        for o_i in range(1, output_features_length):
            
            _input_features = _input_features.to(device)
            
            #print("_input_features s ", _input_features.shape)
            
            _pred_features = decoder(_input_features)
            _pred_features = torch.unsqueeze(_pred_features, axis=1)

            _input_features = _input_features[:, 1:, :].detach().clone()
            _pred_features = _pred_features.detach().clone()
            
            pred_features.append(_pred_features.cpu())
            
            _input_features = torch.cat((_input_features, _pred_features), axis=1)
                
            #print("_input_features s ", _input_features.shape)
            
    pred_features = torch.cat(pred_features, axis=1)
    pred_features = torch.permute(pred_features, (0, 2, 1))
    pred_audio = vocos.decode(pred_features)
    
    torchaudio.save(file_name, pred_audio.detach().cpu(), audio_sample_rate)

### Perform Audio Continuation

In [ ]:
audio_file = "../../../../Data/Audio/Gutenberg/Night_and_Day_by_Virginia_Woolf_48khz.wav"
audio_start_time_sec = 60.0
audio_end_time_sec = 100.0

audio_file_gui = widgets.Text(value=audio_file, description="Audio File:", style={'description_width': 'initial'})
audio_start_time_sec_gui = widgets.FloatText(value=audio_start_time_sec, description="Audio Start Time [Seconds]:", style={'description_width': 'initial'})
audio_end_time_sec_gui = widgets.FloatText(value=audio_end_time_sec, description="Audio End Time [Seconds]", style={'description_width': 'initial'})

display(audio_file_gui)
display(audio_start_time_sec_gui)
display(audio_end_time_sec_gui)

In [ ]:
audio_file = audio_file_gui.value
audio_start_time_sec = audio_start_time_sec_gui.value
audio_end_time_sec = audio_end_time_sec_gui.value

In [ ]:
waveform_data, _ = torchaudio.load(audio_file)

export_orig_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/orig_{}-{}.wav".format(audio_start_time_sec, audio_end_time_sec))
export_ref_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/ref_{}-{}.wav".format(audio_start_time_sec, audio_end_time_sec))
export_pred_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/pred_{}-{}_epoch_{}.wav".format(audio_start_time_sec, audio_end_time_sec, epochs))

In [ ]:
audio_start_time_sec = 0.0
audio_end_time_sec = 20.0

for audio_file_index in range(len(audio_files)):
    audio_file = audio_files[audio_file_index]
    waveform_data, _ = torchaudio.load(audio_file_path + audio_file)

    export_orig_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/orig_{}_{}-{}.wav".format(audio_file, audio_start_time_sec, audio_end_time_sec))
    export_ref_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/ref_{}_{}-{}.wav".format(audio_file, audio_start_time_sec, audio_end_time_sec))
    export_pred_audio(waveform_data, audio_start_time_sec, audio_end_time_sec, "results/audio/pred_{}_{}-{}_epoch_{}.wav".format(audio_file, audio_start_time_sec, audio_end_time_sec, epochs))y